<a href="https://colab.research.google.com/github/Ilvecho/Project_7/blob/main/Project_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The data available for this project are **real data** so they are dirty.

A lot of values are missing, so we need to think how to handle them.

Normally, several hypothesis need to be tested, but in this case we are short in time, so we would make just one iteration of reasonable assumptions.

Furthermore, the data contains the **date** of each step, as well as the date of first and last contact.

We will leverage these pieces of information mainly to fill out some missing values. However, for the model itself we will just consider the binary state (yes - no) rather than the dates. This choice is driven by the need to prioritize simplicity for time constraints.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import os
import pickle
import re
import random

import torch
from torch import nn
from torch import flatten
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam

from google.colab import files,drive
drive.mount('/content/gdrive')

# get the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Mounted at /content/gdrive


In [20]:
data = pd.read_csv('/content/gdrive/MyDrive/P7_files/SalesCRM - CRM.csv')

In [110]:
data.shape

(11032, 13)

In [111]:
for col in data.columns:
  print(col)

ID
Country
Education
First Contact
Last Contact
Status
Stage
First Call
Signed up for a demo
Filled in customer survey
Did sign up to the platform
Account Manager assigned
Subscribed


In [71]:
# Many of the cells have value NaN. Let's replace it with a different value so that we are able to count the number of NaN's as well
parsed_data = data.copy()
parsed_data['Country'] = parsed_data['Country'].fillna('missing')
parsed_data['Education'] = parsed_data['Education'].fillna('missing')
parsed_data['Status'] = parsed_data['Status'].fillna('missing')
parsed_data['Stage'] = parsed_data['Stage'].fillna('missing')


parsed_data.set_index('ID', inplace=True)

# Date features parsing

All the other features represent a date in time, so we need convert the columns to **datetime format**.

Let's define a parser function

In [72]:
# define the datetime parser function
def datetime_parser(column):

  # Iterate through all the rows, because different rows can have different datetime patterns
  for i in list(column.index):

    # Define the patterns observed in the data
    pattern_1 = re.compile(r'^[0-9]{4}\-[0-9]{2}\-[0-9]{4}$')
    pattern_2 = re.compile(r'^[0-9]{2}\.[0-9]{2}\.[0-9]{4}$')
    pattern_3 = re.compile(r'^[0-9]{2}\-[0-9]{2}\-[0-9]{4}$')

    # If the row is already in Datetime format, pass
    if isinstance(column.loc[i], pd.Timestamp):
      pass

    # If the value is missing, assign Not a Time
    elif pd.isna(column.loc[i]):
      column.loc[i] = pd.NaT

    # If the string matches the first patter, convert accordingly
    elif pattern_1.match(column.loc[i]):
      column.loc[i] = pd.to_datetime(column.loc[i], format='%Y-%m-%d')

    # If the string matches the second patter, convert accordingly
    elif pattern_2.match(column.loc[i]):
      column.loc[i] = pd.to_datetime(column.loc[i], format='%d.%m.%Y')

    # If the string matches the third patter, we are still not sure about the format
    elif pattern_3.match(column.loc[i]):
      # So, try one format and catch exceptions and use the other possible format
      try:
        column.loc[i] = pd.to_datetime(column.loc[i], format='%d-%m-%Y')
      except:
        column.loc[i] = pd.to_datetime(column.loc[i], format='%m-%d-%Y')

  return pd.to_datetime(column)

Manually fix an exception

In [73]:
# There is one row that has format Y-d-m rather then Y-m-d
parsed_data['First Contact'].iloc[9737] = "2021-12-13"
parsed_data['Last Contact'].iloc[9737] = "2021-12-13"

Parse the **First Contact** column

In [74]:
parsed_data['First Contact'] = datetime_parser(parsed_data['First Contact'])

Parse the **Last Contact** column

In [75]:
parsed_data['Last Contact'] = datetime_parser(parsed_data['Last Contact'])

<ipython-input-72-3cadf41cfe05>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-72-3cadf41cfe05>:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-72-3cadf41cfe05>:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Parse the **First Call** column

In [76]:
parsed_data['First Call'] = datetime_parser(parsed_data['First Call'])

<ipython-input-72-3cadf41cfe05>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Parse the **Signed up for a demo** column

In [77]:
parsed_data['Signed up for a demo'] = datetime_parser(parsed_data['Signed up for a demo'])

<ipython-input-72-3cadf41cfe05>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Parse the **Filled in customer survey** column

In [78]:
parsed_data['Filled in customer survey'] = datetime_parser(parsed_data['Filled in customer survey'])

<ipython-input-72-3cadf41cfe05>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Parse the **Did sign up to the platform** column.

Actually we notice that some cells in this column do not contain a datetime value, but rather the string "No"

In [79]:
mask = (parsed_data['Did sign up to the platform'] == 'No')
did_not_signed = parsed_data[mask]
did_not_signed['First Call'].isna().sum()

0

Looking at the "First Call" column of these rows, we notice that in **all cases** the first call happened and the potential customer decided not to Sign up to the platform. Hopefully our model will be able to pick up this trend.

Now we need to remove the 'No' values from the column, but we stil want to preserve the information.

Let's create a **new boolean feature**

In [80]:
# first let's rename the original column
parsed_data.rename(columns={'Did sign up to the platform': 'Date Platform sign up'}, inplace=True)
# Create the new column
parsed_data['Bool Platform sign up'] = np.nan

# Create a mask to find the rows where Customer actually did signed up
signed_mask = (parsed_data['Date Platform sign up'].notna()) & (parsed_data['Date Platform sign up'] != 'No')
parsed_data['Bool Platform sign up'][signed_mask] = True

# Create a mask for the rows where we know that customer DID NOT signed up
not_signed_mask = (parsed_data['Date Platform sign up'] == 'No')
parsed_data['Bool Platform sign up'][not_signed_mask] = False

<ipython-input-80-1e7ade2c0171>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Now we can actually proceed to parsing the column 'Date Platform sign up'

In [81]:
parsed_data['Date Platform sign up'] = parsed_data['Date Platform sign up'].replace('No', pd.NaT)

# When running the parser we notice that one cell has a typo in the year: 20221
# let's find the row and fix the issue
idx = np.where(parsed_data['Date Platform sign up'] == '20221-08-24')
parsed_data['Date Platform sign up'].iloc[idx] = '2021-08-24'

<ipython-input-81-4868518e71eb>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [82]:
# Finally parse the column
parsed_data['Date Platform sign up'] = datetime_parser(parsed_data['Date Platform sign up'])

<ipython-input-72-3cadf41cfe05>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [83]:
parsed_data['Account Manager assigned'] = datetime_parser(parsed_data['Account Manager assigned'])

<ipython-input-72-3cadf41cfe05>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [84]:
# Also column "subscribed" has an unexpected value - we need to manually modify it
idx = np.where(parsed_data['Subscribed'] == '0000-00-00')
parsed_data['Subscribed'].iloc[idx] = pd.NaT

<ipython-input-84-5284ecdf84cd>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [85]:
parsed_data['Subscribed'] = datetime_parser(parsed_data['Subscribed'])

<ipython-input-72-3cadf41cfe05>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [86]:
parsed_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11032 entries, 1 to 11793
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Country                    11032 non-null  object        
 1   Education                  11032 non-null  object        
 2   First Contact              10317 non-null  datetime64[ns]
 3   Last Contact               10722 non-null  datetime64[ns]
 4   Status                     11032 non-null  object        
 5   Stage                      11032 non-null  object        
 6   First Call                 441 non-null    datetime64[ns]
 7   Signed up for a demo       283 non-null    datetime64[ns]
 8   Filled in customer survey  184 non-null    datetime64[ns]
 9   Date Platform sign up      296 non-null    datetime64[ns]
 10  Account Manager assigned   71 non-null     datetime64[ns]
 11  Subscribed                 47 non-null     datetime64[ns]
 12  Bool

# Exploratory Data Analysis - Country

In [87]:
# Bar plot for the counties
fig = px.bar(
    parsed_data,
    x=parsed_data.groupby('Country').size().values,
    y=parsed_data.groupby('Country').size().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Country',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

I did not expect these many values for the country. Let's analyze this more in details:

In [88]:
parsed_data['Country'].value_counts()

USA                         6604
missing                      891
Canada                       817
France                       336
UK                           329
                            ... 
uSA                            1
Hong Kong                      1
Singapore                      1
Cameroon                       1
Central African Republic       1
Name: Country, Length: 104, dtype: int64

There are **104** different Country values.
Actually there are some typos (e.g. uSA), so let's elaborate a bit the strings to get a more informative overview

In [89]:
countries = parsed_data['Country'].str.rstrip().str.lower()
countries.value_counts()

usa                         6641
missing                      891
canada                       818
france                       336
uk                           331
                            ... 
full time                      1
bulgaria                       1
senegal                        1
jordan                         1
central african republic       1
Name: Country, Length: 94, dtype: int64

The modification showed indeed some typos. \

We also added a **rstrip** function to remove trailing spaces.

The new count is **94**

In [90]:
countries.value_counts().tail(30)

bolivia                     2
kenya                       2
malaysia                    2
sri lanka                   2
philadelphia                1
congo                       1
california                  1
czechia                     1
russia&ukraine              1
nottingham                  1
korea                       1
bulgaria & uk               1
vietnamese                  1
greek                       1
romania                     1
turkey                      1
latvia                      1
hong kong                   1
seoul                       1
venezuela                   1
chuang                      1
benin                       1
guinea                      1
cameroon                    1
serbia                      1
full time                   1
bulgaria                    1
senegal                     1
jordan                      1
central african republic    1
Name: Country, dtype: int64

Looking at the tail, we still notice some duplicates (non compehensive list):
- Czechia is mentioned twice
- Bulgaria
- Nottingham is a city, not a country. Same is true for Seul, philadelphia and california
- Vietnamese is not a country
- Full time is not a country

Hence, let's create a lookup table to pre-process the country column

In [91]:
lookup_table = {
    "nottingham": "uk",
    "seoul": "south korea",
    "greek": "greece",
    "vietnamese": "vietnam",
    "bulgaria & uk": "bulgaria",
    "korea": "south korea",
    "russia&ukraine": "russia",
    "california": "usa",
    "philadelphia": "usa",
    "ca": "usa",
    'england': "uk",
    "dubai": "united arab emirates",
    "czechia (czech republic)": "czechia",
    "chuang": "china",
    "-": "missing"
}

In [92]:
countries = countries.apply(lambda x: lookup_table[x] if x in lookup_table.keys() else x)
countries.value_counts()

usa                         6645
missing                      901
canada                       818
france                       336
uk                           334
                            ... 
benin                          1
cameroon                       1
full time                      1
turkey                         1
central african republic       1
Name: Country, Length: 79, dtype: int64

Now we have **79** unique values

In [93]:
parsed_data['Country'] = countries.copy()

# Bar plot for the counties
fig = px.bar(
    parsed_data,
    x=countries.value_counts(),
    y=countries.value_counts().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Country',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

The feature 'Country' will be part of the **state**:

we will **one-hot encode** the countries to obtain the first part of the state vector (size 79).

Note that this part of the state **cannot be changed** by any of the actions: it is a bio information

In [94]:
idx = np.where(parsed_data['Country'] == 'missing')
print(f'Percentage of Missing values in Column Country: {idx[0].shape[0]/parsed_data.shape[0] * 100 :.2f} %')

Percentage of Missing values in Column Country: 8.17 %


All the rows of this column must have a value - the value 'Missing' does not make sense.

Since we have no additional information, we decide to fill out the missing values with the **10 most observed categories** in the observed proportion.

In [95]:
# Select the top 10 observed categories
top_categories = parsed_data['Country'].value_counts().nlargest(10).index

# Filter the data to include only the top 10 categories
parsed_data_top_categories = parsed_data[parsed_data['Country'].isin(top_categories)]
# Drop the Missing
parsed_data_top_categories = parsed_data_top_categories[parsed_data_top_categories['Country'] != 'missing']

missing_values_count = idx[0].shape[0]

# Replace 'Missing' values using observed categories in proportion (top 10)
replacement_values = parsed_data_top_categories['Country'].sample(
    n=missing_values_count,
    replace=True,
    weights=parsed_data_top_categories.groupby('Country')['Country'].transform('count')
).values

# Replace the missing values
parsed_data['Country'].iloc[idx[0]] = replacement_values

# Bar plot for the counties
fig = px.bar(
    parsed_data,
    x=parsed_data['Country'].value_counts(),
    y=parsed_data['Country'].value_counts().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Country',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

<ipython-input-95-f55ed61685ec>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Exploratory Data Analysis - Education

In [96]:
# Bar plot for the counties
fig = px.bar(
    parsed_data,
    x=parsed_data.groupby('Education').size().values,
    y=parsed_data.groupby('Education').size().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Education',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

the education category has a limited number of options - good.

Still important to notice that a lot of samples have value **missing**.

In [97]:
parsed_data['Education'].value_counts()

missing    3603
B27        1942
B11         923
B10         670
B9          635
B1          393
B29         353
B21         331
B17         227
B14         218
B30         218
B16         213
B19         168
B28         132
B8          129
B12          93
B15          87
B13          77
B25          76
B24          70
B22          69
B26          62
B23          57
B18          57
B3           42
B20          39
B2           35
B6           34
B4           33
B5           27
B7           19
Name: Education, dtype: int64

In [98]:
len(parsed_data['Education'].value_counts())

31

The feature 'Education' will be also part of the **state**:

we will **one-hot encode** the education levels to obtain the second part of the state vector (size 31).

Note that this part of the state **cannot be changed** by any of the actions: it is a bio information

In [99]:
idx = np.where(parsed_data['Education'] == 'missing')
print(f'Percentage of Missing values in Column Education: {idx[0].shape[0]/parsed_data.shape[0] * 100 :.2f} %')

Percentage of Missing values in Column Education: 32.66 %


All the rows of this column must have a value - the value 'Missing' does not make sense.

Since we have no additional information, we decide to fill out the missing values with all the possible categories the in the observed proportion.

In [100]:
# Select the top 10 observed categories
top_categories = parsed_data['Education'].value_counts().index

# Drop the Missing
parsed_data_no_missing = parsed_data[parsed_data['Education'] != 'missing']

missing_values_count = idx[0].shape[0]

# Replace 'Missing' values using observed categories in proportion (top 10)
replacement_values = parsed_data_no_missing['Education'].sample(
    n=missing_values_count,
    replace=True,
    weights=parsed_data_no_missing.groupby('Education')['Education'].transform('count')
).values

# Replace the missing values
parsed_data['Education'].iloc[idx[0]] = replacement_values

# Bar plot for the counties
fig = px.bar(
    parsed_data,
    x=parsed_data['Education'].value_counts(),
    y=parsed_data['Education'].value_counts().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Education',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

<ipython-input-100-14b2461292a4>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Exploratory Data Analysis - Status

The **Status** column, formerly "Message State", indicates how many times a potential customer was contacted. If the customer at some point is interested, they will proceed with the demo call

In [101]:
fig = px.bar(
    parsed_data,
    x=parsed_data.groupby('Status').size().values,
    y=parsed_data.groupby('Status').size().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Status',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

Also the status will be part of the **state space**:

We will encode it as the Integer number of messages sent to the potential client.

Note that this part of the state **can be changed** via an action

In [102]:
idx = np.where(parsed_data['Status'] == 'missing')
parsed_data.iloc[idx[0]]

,Country,Education,First Contact,Last Contact,Status,Stage,First Call,Signed up for a demo,Filled in customer survey,Date Platform sign up,Account Manager assigned,Subscribed,Bool Platform sign up
ID,,,,,,,,,,,,,
1,usa,B27,NaT,NaT,missing,missing,NaT,NaT,NaT,2022-04-27,NaT,NaT,True
2,austria,B27,NaT,NaT,missing,missing,2022-04-28,2022-04-25,2022-04-25,2022-04-25,NaT,NaT,True
3,united arab emirates,B27,NaT,NaT,missing,missing,NaT,2022-04-24,NaT,NaT,NaT,NaT,NaN
4,france,B1,NaT,NaT,missing,missing,2022-04-22,2022-04-20,2022-04-20,2022-04-22,2022-04-22,NaT,True
5,usa,B9,NaT,NaT,missing,missing,2022-04-23,2022-04-19,2022-04-19,NaT,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9421,usa,B1,NaT,2021-09-01,missing,missing,NaT,NaT,NaT,NaT,NaT,NaT,NaN
9422,usa,B27,NaT,2021-09-01,missing,missing,NaT,NaT,NaT,NaT,NaT,NaT,NaN
9423,mexico,B9,NaT,2021-09-01,missing,missing,2021-09-30,2021-12-22,2021-12-22,2021-12-22,NaT,NaT,True


Looking at the rows with Missing values, we can see that some of them show later steps in the onboarding process.

Hence, we are going to deal with the Status missing values as follow:
- If the 'First Contact' or 'Last Contact' columns have a value, we are going to fill in the missing Status value
- If the 'Stage' has a value, we are going to fill 1st message
- If any of the onboarding steps have been done, then  we will change the status to 1st message.

In [104]:
mask_contact = parsed_data['First Contact'].notna() | parsed_data['Last Contact'].notna() | (parsed_data['Stage'] != 'missing')
mask_onboarding = (parsed_data['First Call'].notna() | parsed_data['Signed up for a demo'].notna() | parsed_data['Filled in customer survey'].notna()
                  | parsed_data['Date Platform sign up'].notna() | parsed_data['Account Manager assigned'].notna() | parsed_data['Subscribed'].notna())
mask_missing = parsed_data['Status'] == 'missing'
mask = (mask_contact | mask_onboarding) & mask_missing
parsed_data['Status'][mask]

ID
1        missing
2        missing
3        missing
4        missing
5        missing
          ...   
9421     missing
9422     missing
9423     missing
9830     missing
11154    missing
Name: Status, Length: 799, dtype: object

In [105]:
parsed_data['Status'][mask] = '1st message'

fig = px.bar(
    parsed_data,
    x=parsed_data.groupby('Status').size().values,
    y=parsed_data.groupby('Status').size().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Status',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

<ipython-input-105-049dd4476971>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [106]:
parsed_data[parsed_data['Status'] == 'missing']

,Country,Education,First Contact,Last Contact,Status,Stage,First Call,Signed up for a demo,Filled in customer survey,Date Platform sign up,Account Manager assigned,Subscribed,Bool Platform sign up
ID,,,,,,,,,,,,,
126,canada,B27,NaT,NaT,missing,missing,NaT,NaT,NaT,NaT,NaT,NaT,NaN
191,usa,B28,NaT,NaT,missing,missing,NaT,NaT,NaT,NaT,NaT,NaT,NaN
192,usa,B10,NaT,NaT,missing,missing,NaT,NaT,NaT,NaT,NaT,NaT,NaN
278,canada,B27,NaT,NaT,missing,missing,NaT,NaT,NaT,NaT,NaT,NaT,NaN


The 4 remaining missing values will be considered as **never contacted**

# Exploratory Data Analysis - Stage

The column stage reflects the potential customer reaction before and/or after the demo call

In [107]:
fig = px.bar(
    parsed_data,
    x=parsed_data.groupby('Stage').size().values,
    y=parsed_data.groupby('Stage').size().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Stage',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

The feature 'Stage' will be also part of the **state**:

we will **one-hot encode** the education levels to obtain the second part of the state vector (size 7).

Note that this part of the state **can be changed**: indeed it starts empty and based on previous actions it can be filled out

For simplicity sake, we can **group** the categories in order to have just four:
- missing (i.e. unknown)
- not interested
- interested
- subscribed already

In [108]:
mask = ((parsed_data['Stage'] == 'not interested') | (parsed_data['Stage'] == 'do not contact') | (parsed_data['Stage'] == 'did not join the call') | (parsed_data['Stage'] == 'declined/canceled call') )
parsed_data['Stage'][mask] = 'not interested'

fig = px.bar(
    parsed_data,
    x=parsed_data.groupby('Stage').size().values,
    y=parsed_data.groupby('Stage').size().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Stage',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

<ipython-input-108-ae362839a44d>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



check if amongst the not interested, some have done the onboarding steps

In [115]:
idx = np.where(parsed_data['Stage'] == 'not interested')
tmp = parsed_data.iloc[idx]

tmp['First Call'].notna().sum()

126

We notice that some of the 'not interested' potential customers actually had a first call.

This indicates that the column Stage is updated also **during the onboarding process**

This means that it would be **very hard** to fill out the missing values with the available information

There is one thing we can do though:

since we understood that the feature Stage is updated also during the onboarding process, we can use its values 'subscribed already' to fill out **missing subscription dates**

In [117]:
mask = (parsed_data['Stage'] == 'subscribed already') & parsed_data['Subscribed'].isna()
parsed_data['Subscribed'][mask] = pd.to_datetime('2023-11-20')
parsed_data[mask]

<ipython-input-117-8f96c85a1e7b>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Country,Education,First Contact,Last Contact,Status,Stage,First Call,Signed up for a demo,Filled in customer survey,Date Platform sign up,Account Manager assigned,Subscribed,Bool Platform sign up
ID,,,,,,,,,,,,,
170,usa,B10,NaT,NaT,1st message,subscribed already,2021-11-26,2021-11-24,2021-11-24,2021-11-24,2021-12-01,2023-11-20,True
213,canada,B27,NaT,NaT,1st message,subscribed already,2021-11-03,2021-10-26,2021-10-26,2021-11-06,2021-11-06,2023-11-20,True
7889,usa,B27,2021-05-15,2021-05-15,1st message,subscribed already,2021-07-08,NaT,NaT,2021-07-08,2021-08-02,2023-11-20,True


# Date features analysis - First and Last contact

let's check the order of the dates.

First we focus on the **First Contact** and **Last Contact** columns.

We expect the Last contact to be **greater or equal** to the First Contact, but as we can see there are (very few) outliers

In [38]:
print('Number of rows in which the First Contact happened BEFORE the Last contact: ', (parsed_data['First Contact'] <= parsed_data['Last Contact']).sum())
print('Number of rows in which the First Contact happened AFTER the Last contact: ', (parsed_data['First Contact'] > parsed_data['Last Contact']).sum())

Number of rows in which the First Contact happened BEFORE the Last contact:  10292
Number of rows in which the First Contact happened AFTER the Last contact:  3


Then, we would like to understand the **order** of the Onboarding steps, which are:
- First Call
- Signed up for a demo
- Filled in customer survey
- Did sign up to the platform
- Account Manager assigned
- Subscribed

# Date features analysis - First Call

In [42]:
col_1 = 'First Call'
col_2 = 'Signed up for a demo'

print(f'Number of rows in which the {col_1} happened BEFORE the {col_2}: ', (parsed_data[col_1] < parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AT THE SAME TIME as the {col_2}: ', (parsed_data[col_1] == parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AFTER the {col_2}: ', (parsed_data[col_1] > parsed_data[col_2]).sum())

Number of rows in which the First Call happened BEFORE the Signed up for a demo:  10
Number of rows in which the First Call happened AT THE SAME TIME as the Signed up for a demo:  9
Number of rows in which the First Call happened AFTER the Signed up for a demo:  96


In [43]:
col_1 = 'First Call'
col_2 = 'Filled in customer survey'

print(f'Number of rows in which the {col_1} happened BEFORE the {col_2}: ', (parsed_data[col_1] < parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AT THE SAME TIME as the {col_2}: ', (parsed_data[col_1] == parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AFTER the {col_2}: ', (parsed_data[col_1] > parsed_data[col_2]).sum())

Number of rows in which the First Call happened BEFORE the Filled in customer survey:  10
Number of rows in which the First Call happened AT THE SAME TIME as the Filled in customer survey:  10
Number of rows in which the First Call happened AFTER the Filled in customer survey:  93


In [44]:
col_1 = 'First Call'
col_2 = 'Did sign up to the platform'

print(f'Number of rows in which the {col_1} happened BEFORE the {col_2}: ', (parsed_data[col_1] < parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AT THE SAME TIME as the {col_2}: ', (parsed_data[col_1] == parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AFTER the {col_2}: ', (parsed_data[col_1] > parsed_data[col_2]).sum())

Number of rows in which the First Call happened BEFORE the Did sign up to the platform:  29
Number of rows in which the First Call happened AT THE SAME TIME as the Did sign up to the platform:  151
Number of rows in which the First Call happened AFTER the Did sign up to the platform:  57


In [45]:
col_1 = 'First Call'
col_2 = 'Account Manager assigned'

print(f'Number of rows in which the {col_1} happened BEFORE the {col_2}: ', (parsed_data[col_1] < parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AT THE SAME TIME as the {col_2}: ', (parsed_data[col_1] == parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AFTER the {col_2}: ', (parsed_data[col_1] > parsed_data[col_2]).sum())

Number of rows in which the First Call happened BEFORE the Account Manager assigned:  60
Number of rows in which the First Call happened AT THE SAME TIME as the Account Manager assigned:  9
Number of rows in which the First Call happened AFTER the Account Manager assigned:  0


In [46]:
col_1 = 'First Call'
col_2 = 'Subscribed'

print(f'Number of rows in which the {col_1} happened BEFORE the {col_2}: ', (parsed_data[col_1] < parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AT THE SAME TIME as the {col_2}: ', (parsed_data[col_1] == parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AFTER the {col_2}: ', (parsed_data[col_1] > parsed_data[col_2]).sum())

Number of rows in which the First Call happened BEFORE the Subscribed:  46
Number of rows in which the First Call happened AT THE SAME TIME as the Subscribed:  0
Number of rows in which the First Call happened AFTER the Subscribed:  1


It is hard to draw a conclusion on the order of the first steps.

However, we can confidently say that the First Call surely happens **before**:
- Account Manager assigned
- Subscribed (in this case there is one outlier, we will need to handle it)

# Date features analysis - Signed up for a demo

In [47]:
col_1 = 'Signed up for a demo'
col_2 = 'Filled in customer survey'

print(f'Number of rows in which the {col_1} happened BEFORE the {col_2}: ', (parsed_data[col_1] < parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AT THE SAME TIME as the {col_2}: ', (parsed_data[col_1] == parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AFTER the {col_2}: ', (parsed_data[col_1] > parsed_data[col_2]).sum())

Number of rows in which the Signed up for a demo happened BEFORE the Filled in customer survey:  71
Number of rows in which the Signed up for a demo happened AT THE SAME TIME as the Filled in customer survey:  113
Number of rows in which the Signed up for a demo happened AFTER the Filled in customer survey:  0


In [48]:
col_1 = 'Signed up for a demo'
col_2 = 'Did sign up to the platform'

print(f'Number of rows in which the {col_1} happened BEFORE the {col_2}: ', (parsed_data[col_1] < parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AT THE SAME TIME as the {col_2}: ', (parsed_data[col_1] == parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AFTER the {col_2}: ', (parsed_data[col_1] > parsed_data[col_2]).sum())

Number of rows in which the Signed up for a demo happened BEFORE the Did sign up to the platform:  48
Number of rows in which the Signed up for a demo happened AT THE SAME TIME as the Did sign up to the platform:  37
Number of rows in which the Signed up for a demo happened AFTER the Did sign up to the platform:  3


In [49]:
col_1 = 'Signed up for a demo'
col_2 = 'Account Manager assigned'

print(f'Number of rows in which the {col_1} happened BEFORE the {col_2}: ', (parsed_data[col_1] < parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AT THE SAME TIME as the {col_2}: ', (parsed_data[col_1] == parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AFTER the {col_2}: ', (parsed_data[col_1] > parsed_data[col_2]).sum())

Number of rows in which the Signed up for a demo happened BEFORE the Account Manager assigned:  47
Number of rows in which the Signed up for a demo happened AT THE SAME TIME as the Account Manager assigned:  0
Number of rows in which the Signed up for a demo happened AFTER the Account Manager assigned:  0


In [50]:
col_1 = 'Signed up for a demo'
col_2 = 'Subscribed'

print(f'Number of rows in which the {col_1} happened BEFORE the {col_2}: ', (parsed_data[col_1] < parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AT THE SAME TIME as the {col_2}: ', (parsed_data[col_1] == parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AFTER the {col_2}: ', (parsed_data[col_1] > parsed_data[col_2]).sum())

Number of rows in which the Signed up for a demo happened BEFORE the Subscribed:  32
Number of rows in which the Signed up for a demo happened AT THE SAME TIME as the Subscribed:  0
Number of rows in which the Signed up for a demo happened AFTER the Subscribed:  0


Looking at the above results, it seems that the **Signed up for a demo** step needs to happen before (or on the same day as) the next steps:
- Filled in customer survey
- Did sign up to the platform (in this case there are three outliers)
- Account Manager assigned
- Subscribed

# Date features analysis - Filled in customer survey

In [51]:
col_1 = 'Filled in customer survey'
col_2 = 'Did sign up to the platform'

print(f'Number of rows in which the {col_1} happened BEFORE the {col_2}: ', (parsed_data[col_1] < parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AT THE SAME TIME as the {col_2}: ', (parsed_data[col_1] == parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AFTER the {col_2}: ', (parsed_data[col_1] > parsed_data[col_2]).sum())

Number of rows in which the Filled in customer survey happened BEFORE the Did sign up to the platform:  37
Number of rows in which the Filled in customer survey happened AT THE SAME TIME as the Did sign up to the platform:  48
Number of rows in which the Filled in customer survey happened AFTER the Did sign up to the platform:  2


In [52]:
col_1 = 'Filled in customer survey'
col_2 = 'Account Manager assigned'

print(f'Number of rows in which the {col_1} happened BEFORE the {col_2}: ', (parsed_data[col_1] < parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AT THE SAME TIME as the {col_2}: ', (parsed_data[col_1] == parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AFTER the {col_2}: ', (parsed_data[col_1] > parsed_data[col_2]).sum())

Number of rows in which the Filled in customer survey happened BEFORE the Account Manager assigned:  47
Number of rows in which the Filled in customer survey happened AT THE SAME TIME as the Account Manager assigned:  0
Number of rows in which the Filled in customer survey happened AFTER the Account Manager assigned:  0


In [53]:
col_1 = 'Filled in customer survey'
col_2 = 'Subscribed'

print(f'Number of rows in which the {col_1} happened BEFORE the {col_2}: ', (parsed_data[col_1] < parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AT THE SAME TIME as the {col_2}: ', (parsed_data[col_1] == parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AFTER the {col_2}: ', (parsed_data[col_1] > parsed_data[col_2]).sum())

Number of rows in which the Filled in customer survey happened BEFORE the Subscribed:  32
Number of rows in which the Filled in customer survey happened AT THE SAME TIME as the Subscribed:  0
Number of rows in which the Filled in customer survey happened AFTER the Subscribed:  0


Once again, it seems that the customer survey is filled before the next steps - except for the Platform signup that has 2 outliers

# Date features analysis - Did sign up to the platform

In [54]:
col_1 = 'Did sign up to the platform'
col_2 = 'Account Manager assigned'

print(f'Number of rows in which the {col_1} happened BEFORE the {col_2}: ', (parsed_data[col_1] < parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AT THE SAME TIME as the {col_2}: ', (parsed_data[col_1] == parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AFTER the {col_2}: ', (parsed_data[col_1] > parsed_data[col_2]).sum())

Number of rows in which the Did sign up to the platform happened BEFORE the Account Manager assigned:  62
Number of rows in which the Did sign up to the platform happened AT THE SAME TIME as the Account Manager assigned:  7
Number of rows in which the Did sign up to the platform happened AFTER the Account Manager assigned:  0


In [55]:
col_1 = 'Did sign up to the platform'
col_2 = 'Subscribed'

print(f'Number of rows in which the {col_1} happened BEFORE the {col_2}: ', (parsed_data[col_1] < parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AT THE SAME TIME as the {col_2}: ', (parsed_data[col_1] == parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AFTER the {col_2}: ', (parsed_data[col_1] > parsed_data[col_2]).sum())

Number of rows in which the Did sign up to the platform happened BEFORE the Subscribed:  43
Number of rows in which the Did sign up to the platform happened AT THE SAME TIME as the Subscribed:  2
Number of rows in which the Did sign up to the platform happened AFTER the Subscribed:  0


# Date features analysis - Account Manager assigned

In [56]:
col_1 = 'Account Manager assigned'
col_2 = 'Subscribed'

print(f'Number of rows in which the {col_1} happened BEFORE the {col_2}: ', (parsed_data[col_1] < parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AT THE SAME TIME as the {col_2}: ', (parsed_data[col_1] == parsed_data[col_2]).sum())
print(f'Number of rows in which the {col_1} happened AFTER the {col_2}: ', (parsed_data[col_1] > parsed_data[col_2]).sum())

Number of rows in which the Account Manager assigned happened BEFORE the Subscribed:  30
Number of rows in which the Account Manager assigned happened AT THE SAME TIME as the Subscribed:  3
Number of rows in which the Account Manager assigned happened AFTER the Subscribed:  11


# Conclusions

It seems that the order of the steps is not clearly defined. However we can conclude that:
- 'First Call' and 'Signed up for a demo' are the **first steps**
- There is 'Filled in customer survey' as **second step**
- Then, 'Did sign up to the platform' is the **third step**
- Lastly, 'Account Manager assigned' and 'Subscribed' are the **last steps**

The onboarding steps will also be part of the **state**. We will encode them as follow:

a (6,) binary vector that indicates whether a step was done or not;

- The starting point is [0,0,0,0,0,0]
- [1,0,0,0,0,0] and [0,1,0,0,0,0] are both valid first steps
- [1,1,0,0,0,0] is needed to move to [1,1,1,0,0,0]
- [1,1,1,0,0,0] is needed to move to [1,1,1,1,0,0]
- From [1,1,1,1,0,0] it is possible to go in both [1,1,1,1,1,0] and [1,1,1,1,0,1]
- [1,1,1,1,1,1] is the desired state

To summarize, the state will be made of the **concatenation** of the following arrays:

- One-hot encoding of the Country - **cannot** be changed - len 79
- One-hot encoding of the Education - **cannot** be changed - len 31
- Int for the Status - it can change - len 1
- One-hot encoding of the Stage - it will change from empty to one of the values - len 6 (we want to drop the missing in this encoding. It will be represented by the empty array)
- Binary encoding of the Onboarding steps - it can change - len 6

Total length of the state is: 79 + 31 + 1 + 6 + 6 = 123